### Load tensorflow

In [0]:
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
project_path="/content/dirve/DeepLearn/R6_internal/"

In [4]:
tf.version

<module 'tensorflow._api.v1.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v1/version/__init__.py'>

### Enable Eager Execution if you are using tensorflow 1.x

In [5]:
tf.__version__

'1.14.0'

In [0]:
tf.enable_eager_execution()

### Collect Data

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('./prices.csv')

### Check all columns in the dataset

In [9]:
data.shape

(851264, 7)

In [10]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data=data.drop(['date', 'symbol'],axis=1)

In [0]:
cols=data.columns

In [13]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
df=data.head(1000)

### Convert Float64 to Float32

In [0]:
df=df.astype("float32")

### Divide the data into train and test sets

In [0]:
y = df["close"]
X=df.drop(["close"],axis=1)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=123)

In [18]:
X_train.head()

,open,low,high,volume
498,20.530001,20.440001,20.650000,12718800.0
243,124.330002,123.419998,124.489998,651500.0
314,51.860001,51.820000,52.549999,1234200.0
202,126.339996,126.040001,127.250000,496600.0
300,158.990005,157.710007,159.389999,408300.0


### Normalize Train and Test Data 

In [0]:
from sklearn.preprocessing import Normalizer

In [0]:
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)

In [21]:
X_train

array([[1.6141461e-06, 1.6070699e-06, 1.6235808e-06, 1.0000000e+00],
       [1.9083651e-04, 1.8943973e-04, 1.9108210e-04, 9.9999988e-01],
       [4.2019121e-05, 4.1986710e-05, 4.2578187e-05, 1.0000000e+00],
       ...,
       [2.3780945e-05, 2.3505878e-05, 2.4289406e-05, 1.0000000e+00],
       [1.7972880e-05, 1.7853366e-05, 1.8119972e-05, 1.0000000e+00],
       [3.1503550e-05, 3.1358217e-05, 3.1951997e-05, 1.0000000e+00]],
      dtype=float32)

In [0]:
import numpy as np
y_train=np.array(y_train)

## Building the graph in tensorflow

2.Define Weights and Bias

In [23]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))
w,b

(<tf.Tensor: id=2, shape=(4, 1), dtype=float32, numpy=
 array([[0.],
        [0.],
        [0.],
        [0.]], dtype=float32)>,
 <tf.Tensor: id=5, shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>)

3.Prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y


4.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    with tf.GradientTape() as t:
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Execute the Graph for 100 epochs and observe the loss

In [27]:
for i in range(100):
    
    w, b = train(X_train, y_train, w, b)
    print('Current Loss on iteration', i, loss(y_train, prediction(X_train, w, b)).numpy())

Current Loss on iteration 0 8343.2705
Current Loss on iteration 1 8001.953
Current Loss on iteration 2 7687.3936
Current Loss on iteration 3 7397.495
Current Loss on iteration 4 7130.319
Current Loss on iteration 5 6884.0967
Current Loss on iteration 6 6657.1743
Current Loss on iteration 7 6448.0425
Current Loss on iteration 8 6255.311
Current Loss on iteration 9 6077.683
Current Loss on iteration 10 5913.987
Current Loss on iteration 11 5763.12
Current Loss on iteration 12 5624.086
Current Loss on iteration 13 5495.95
Current Loss on iteration 14 5377.858
Current Loss on iteration 15 5269.025
Current Loss on iteration 16 5168.7266
Current Loss on iteration 17 5076.29
Current Loss on iteration 18 4991.1016
Current Loss on iteration 19 4912.5913
Current Loss on iteration 20 4840.238
Current Loss on iteration 21 4773.5547
Current Loss on iteration 22 4712.099
Current Loss on iteration 23 4655.462
Current Loss on iteration 24 4603.2646
Current Loss on iteration 25 4555.1616
Current Loss o

### Get the shapes and values of W and b

In [28]:
w.shape,b.shape

(TensorShape([Dimension(4), Dimension(1)]), TensorShape([Dimension(1)]))

In [29]:
w

<tf.Tensor: id=5096, shape=(4, 1), dtype=float32, numpy=
array([[2.5196134e-03],
       [2.4950290e-03],
       [2.5409432e-03],
       [3.3785694e+01]], dtype=float32)>

In [30]:
b

<tf.Tensor: id=5099, shape=(1,), dtype=float32, numpy=array([33.785698], dtype=float32)>

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1, input_shape=(4,)))
model.compile(optimizer='sgd', loss='mse')

### Execute the model

In [32]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
700/700 [==============================] - 0s 96us/sample - loss: 6242.6026
Epoch 2/100
700/700 [==============================] - 0s 29us/sample - loss: 4378.9630
Epoch 3/100
700/700 [==============================] - 0s 30us/sample - loss: 4061.7092
Epoch 4/100
700/700 [==============================] - 0s 26us/sample - loss: 4004.7492
Epoch 5/100
700/700 [==============================] - 0s 26us/sample - loss: 3995.2874
Epoch 6/100
700/700 [==============================] - 0s 28us/sample - loss: 3992.9592
Epoch 7/100
700/700 [==============================] - 0s 25us/sample - loss: 3993.6517
Epoch 8/100
700/700 [==============================] - 0s 29us/sample - loss: 3993.0755
Epoch 9/100
700/700 [==============================] - 0s 27us/sample - loss: 3993.4065
Epoch 10/100
700/700 [==============================] - 0s 30us/sample - loss: 3994.6311
Epoch 11/100
700/700 [==============================] - 0s 28us/sample - loss: 3993.5211
Epoch 12/100
700/700 [========

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.get_weights()

[array([[-0.06691001],
        [-0.12559667],
        [ 0.41772893],
        [34.781845  ]], dtype=float32), array([33.788536], dtype=float32)]

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [35]:
df_iris=pd.read_csv("Iris.csv")

df_iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [0]:
X=df_iris.drop(["Species","Id"],axis=1)
y=df_iris["Species"]

In [37]:
X.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
y=le.fit_transform(y)
y=pd.get_dummies(y).values

In [39]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

### Divide the dataset into Training and test (70:30)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=123)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Reshape((784,),input_shape=(28,28,)))
model.add(tf.keras.layers.Reshape((4,),input_shape=(4,)))
model.add(tf.keras.layers.BatchNormalization())

In [0]:
#Add 1st hidden layer
model.add(tf.keras.layers.Dense(10, activation='sigmoid'))

In [0]:
#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(8, activation='sigmoid'))

In [0]:
# output layer
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Fitting the model and predicting 

In [46]:
model.fit(X_train,y_train,         
          validation_data=(X_test,y_test),
          epochs=15)

Train on 105 samples, validate on 45 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/15
105/105 [==============================] - 0s 3ms/sample - loss: 1.5950 - acc: 0.3143 - val_loss: 1.2692 - val_acc: 0.3778
Epoch 2/15
105/105 [==============================] - 0s 119us/sample - loss: 1.5543 - acc: 0.3143 - val_loss: 1.2440 - val_acc: 0.3778
Epoch 3/15
105/105 [==============================] - 0s 111us/sample - loss: 1.5151 - acc: 0.3143 - val_loss: 1.2227 - val_acc: 0.3778
Epoch 4/15
105/105 [==============================] - 0s 112us/sample - loss: 1.4824 - acc: 0.3143 - val_loss: 1.2061 - val_acc: 0.3778
Epoch 5/15
105/105 [==============================] - 0s 109us/sample - loss: 1.4532 - acc: 0.3143 - val_loss: 1.1882 - val_acc: 0.3778
Epoch 6/15
105/105 [==============================] - 0s 110us/sample - loss: 1.4241 - acc: 0.3143 - val_loss: 1.1743 - val_acc: 0.3778
Epoch 7/15
105/105 [===========================

In [47]:
pred = model.predict(X_train)
pred

array([[0.34402168, 0.19587718, 0.46010113],
       [0.34109533, 0.19690153, 0.4620031 ],
       [0.34015167, 0.1935194 , 0.46632898],
       [0.3255445 , 0.19938847, 0.47506702],
       [0.32817575, 0.19706377, 0.47476044],
       [0.33972603, 0.1958306 , 0.46444336],
       [0.3293954 , 0.19740047, 0.47320408],
       [0.32840616, 0.19780932, 0.47378454],
       [0.33722124, 0.19516332, 0.46761546],
       [0.3365875 , 0.19510184, 0.4683106 ],
       [0.33882183, 0.19400504, 0.46717307],
       [0.33694264, 0.19435987, 0.46869743],
       [0.34135062, 0.19524767, 0.4634017 ],
       [0.3367459 , 0.19509122, 0.46816286],
       [0.34167507, 0.19631085, 0.46201405],
       [0.33286884, 0.19363824, 0.47349298],
       [0.33963877, 0.19623475, 0.46412647],
       [0.33484665, 0.19502681, 0.4701266 ],
       [0.32661813, 0.19889227, 0.4744896 ],
       [0.32664683, 0.19844592, 0.47490725],
       [0.33970577, 0.19564216, 0.46465212],
       [0.33723184, 0.19488014, 0.46788803],
       [0.

In [48]:
pred[0]

array([0.34402168, 0.19587718, 0.46010113], dtype=float32)

### Report Accuracy of the predicted values

In [49]:
model.evaluate(X_test,y_test,verbose=0)

[1.101559755537245, 0.37777779]

In [50]:
print("Accuracy is ",model.evaluate(X_test,y_test,verbose=0)[1]*100)
print("Loss is ",model.evaluate(X_test,y_test,verbose=0)[0])

Accuracy is  37.77777850627899
Loss is  1.101559755537245
